Imports

In [2]:
%pip install https://github.com/camlab-bioml/starling/archive/main.zip

import numpy as np
import anndata as ad
import pandas as pd
import torch
from lightning_lite import seed_everything
from pytorch_lightning.callbacks import EarlyStopping  # ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
from starling import starling, utility

seed_everything(10, workers=True)

  Using cached https://github.com/camlab-bioml/starling/archive/main.zip
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 10


10

Set up test dataset path, ADD PATHS TO THE RELEVANT FILES HERE!!!!!!!

In [4]:
number_of_classes = 14
number_of_clusters = 20
test_dataset_patch = 'C:/Users/Dell/Documents/Deep_learning_for_life_science/Projekt_zaliczeniowy/Repo_v3_z_gitem/DL4LS24-IDK/data/test/cell_data.h5ad'
starling_test_patch = 'C:/Users/Dell/Documents/Deep_learning_for_life_science/Projekt_zaliczeniowy/Repo_v3_z_gitem/DL4LS24-IDK/notebooks/model_tuned_on_test_data.pt'
mlp_model_path = 'C:/Users/Dell/Documents/Deep_learning_for_life_science/Projekt_zaliczeniowy/Repo_v3_z_gitem/DL4LS24-IDK/notebooks/MLP_model_trained.pt'

Use STARLING to obtain clusterization of test data and then saveing the model. This takes a long time.

In [ ]:
anndata = ad.read_h5ad(test_dataset_patch)
df=anndata.obs
anndata.X = anndata.layers['exprs'] # FIX!
adata = utility.init_clustering("KM", anndata, k=number_of_clusters)
starling = starling.ST(adata)
log_tb = TensorBoardLogger(save_dir="log")
cb_early_stopping = EarlyStopping(monitor="train_loss", mode="min", verbose=False)
starling.train_and_fit(
    callbacks=[cb_early_stopping],
    logger=[log_tb],
)
torch.save(starling, "STARLING_ON_NEW_DATA.pt")

To obtain this result, I first used starling to obtain clusterization (using code similar to this above), saved it, and then some time later loaded it and proceeded with the rest of this notebook. Now I will load the starling model that I used to obtain the clusters before.

Load starling clusterization on the test dataset


In [3]:
st = torch.load(starling_test_patch)

In [4]:
result = st.result()

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\starling\utility.py:500: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2985.)
  prob_data_given_gamma_d1.T + log_delta[1] - prob_data


In [5]:
result

AnnData object with n_obs × n_vars = 106445 × 40
    obs: 'image', 'sample_id', 'ObjectNumber', 'Pos_X', 'Pos_Y', 'area', 'major_axis_length', 'minor_axis_length', 'eccentricity', 'width_px', 'height_px', 'acquisition_id', 'SlideId', 'Study', 'Box.Description', 'Position', 'SampleId', 'Indication', 'BatchId', 'SubBatchId', 'ROI', 'ROIonSlide', 'includeImage', 'flag_no_cells', 'flag_no_ROI', 'flag_total_area', 'flag_percent_covered', 'small_cell', 'celltypes', 'flag_tumor', 'PD1_pos', 'Ki67_pos', 'cleavedPARP_pos', 'GrzB_pos', 'tumor_patches', 'distToCells', 'CD20_patches', 'Batch', 'cell_labels', 'init_label', 'st_label', 'doublet_prob', 'doublet', 'max_assign_prob'
    var: 'channel', 'use_channel', 'marker'
    uns: 'init_cell_size_centroids', 'init_cell_size_variances', 'st_cell_size_centroids'
    obsm: 'assignment_prob_matrix', 'gamma_assignment_prob_matrix'
    varm: 'init_exp_centroids', 'init_exp_variances', 'st_exp_centroids'
    layers: 'counts', 'exprs'

Prepare data

In [6]:
def prepare_data_for_mlp_prediction(anndata):
    df=anndata.obs
    counts=anndata.layers['counts']
    exprs=anndata.layers['exprs']
    exprs=anndata.layers['exprs']
    list_cols_counts=['counts'+str(i) for i in range(40)]
    list_cols_exprs=['exprs'+str(i) for i in range(40)]
    df_counts = pd.DataFrame(data=counts, columns=list_cols_counts,index=df.index)
    df_exprs = pd.DataFrame(data=exprs, columns=list_cols_exprs,index=df.index)
    df_counts.shape
    df1=df.copy()
    result = pd.concat([df1, df_exprs], axis=1)
    df=result.copy()
    cols_to_remove = ['image', 'sample_id', 'ObjectNumber', 'Pos_X', 'Pos_Y', 'width_px',
        'height_px', 'acquisition_id', 'SlideId', 'Study', 'Box.Description',
        'Position', 'SampleId', 'Indication', 'BatchId', 'SubBatchId', 'ROI',
        'ROIonSlide', 'includeImage', 'flag_no_cells', 'flag_no_ROI',
        'flag_total_area', 'flag_percent_covered', 'small_cell', 'celltypes',
        'flag_tumor', 'PD1_pos', 'Ki67_pos', 'cleavedPARP_pos', 'GrzB_pos',
        'tumor_patches', 'distToCells', 'CD20_patches', 'Batch', 'init_label',
          'doublet_prob', 'doublet', 'max_assign_prob']  # List of columns to remove

    # Remove columns
    df = df.drop(cols_to_remove, axis=1)
    y=df['cell_labels']
    encoder = LabelEncoder()
    y_encoded1 = encoder.fit_transform(y)
    df_y = pd.DataFrame(data=y_encoded1, columns=['target'],index=df.index)
    # print(df_y.shape, df_y, df['cell_labels'])
    result = pd.concat([df, df_y], axis=1)
    #df=result.copy()
    #df = df.drop(['cell_labels'], axis=1)
    df = result.copy()
    df = df.drop(['cell_labels'], axis=1)
    grouped_df = df.groupby('st_label').mean().reset_index()
    grouped_df = grouped_df.drop(['st_label', 'target'], axis=1)
    return grouped_df

result_procesed_for_mlp = prepare_data_for_mlp_prediction(result)

In [7]:
result_procesed_for_mlp

,area,major_axis_length,minor_axis_length,eccentricity,exprs0,exprs1,exprs2,exprs3,exprs4,exprs5,...,exprs30,exprs31,exprs32,exprs33,exprs34,exprs35,exprs36,exprs37,exprs38,exprs39
0,62.514160,10.314495,7.086642,0.662477,0.171345,1.981063,0.887566,0.209427,0.157327,0.826045,...,0.350416,0.327540,0.715831,1.646016,1.444289,0.464353,0.260989,0.073459,3.955299,4.516439
1,67.169233,10.752178,7.719623,0.646200,0.283802,3.055253,0.791620,1.462720,0.777108,3.259295,...,1.418045,1.489223,2.462947,3.123612,1.808267,1.279460,1.686542,0.175130,4.854049,5.416948
2,71.868624,11.130650,7.951923,0.646707,0.313211,3.311560,0.399782,0.725642,0.940417,1.396379,...,0.847548,1.898309,1.396772,1.917929,4.164186,1.038761,0.487544,0.422536,5.179345,5.727672
3,109.587343,13.735212,9.737893,0.657028,0.166030,2.572143,0.138949,0.326985,0.270514,1.445019,...,0.701658,0.537715,1.281391,2.066741,3.589877,0.858009,0.518891,0.198581,4.252582,4.823722
4,99.491217,12.999555,9.387485,0.645023,0.322438,3.148898,0.168708,2.081442,0.801388,3.207447,...,2.026299,1.913475,1.898497,3.747165,2.695493,1.430068,0.852466,0.239343,4.666616,5.228149
5,61.997941,10.225246,7.567560,0.623656,0.238111,2.581572,0.358374,0.545481,0.721509,3.429761,...,1.467186,2.053023,2.901597,2.188091,1.463637,0.989805,0.868347,0.118122,4.801882,5.355838
6,63.653632,10.308632,7.382678,0.640328,0.239323,2.141582,0.534751,1.145918,0.254200,3.101474,...,0.853869,1.033973,1.659677,2.846267,1.445186,0.712720,1.254342,0.119631,4.020344,4.580644
7,110.548277,13.714823,9.869659,0.652787,0.187414,3.494610,0.295291,0.355137,0.362456,1.497204,...,0.782959,0.852379,1.515137,2.554490,4.147365,1.280615,0.575344,0.379523,4.842497,5.408966
8,92.179148,12.864516,8.846033,0.673560,0.178743,1.934967,0.049267,0.207446,0.134377,1.846609,...,0.324707,0.593140,0.692000,1.055570,3.641137,0.229869,0.225700,0.124355,3.918111,4.473303
9,100.564986,13.253661,9.246752,0.669634,0.390248,3.562798,0.403738,1.050485,1.049386,1.934958,...,1.179711,1.122726,1.994809,3.434771,4.700549,1.698820,1.007582,0.451359,5.027208,5.591759


Load trained mlp model

In [5]:
class Linear_block(nn.Module):
    def __init__(self, a, b, dropout):
        super().__init__()
        self.linear = nn.Linear(a, b)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout, inplace=False)
        self.batchnorm = nn.BatchNorm1d(b)
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.batchnorm(x)
        x = self.dropout(x)
        return x


class MLP(nn.Module):
    def __init__(self, layers_list, dropout):
        super().__init__()
        self.my_modules = nn.ModuleList([Linear_block(layers_list[i-1], layers_list[i], dropout) for i in range(1, len(layers_list))])
        self.clasification_head = nn.Linear(layers_list[-1], 14)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        for module in self.my_modules:
            x = module(x)
        x = self.clasification_head(x)
        return x

model = torch.load(mlp_model_path)

Clasify starling clusters using loaded mlp model

In [9]:
def use_mlp_model_to_ger_predictions(model, tensor):
    with torch.no_grad():
        prediction_probabilities = model(tensor)
        predictions = torch.argmax(prediction_probabilities, dim = 1)
        prediction_probabilities = torch.nn.functional.softmax(prediction_probabilities, dim=1)
    return prediction_probabilities, predictions


In [10]:
result_procesed_for_mlp_tensor = torch.tensor(result_procesed_for_mlp.values).float()

In [11]:
prediction_probabilities, predictions = use_mlp_model_to_ger_predictions(model, result_procesed_for_mlp_tensor)

Adding the target column into the result dataframe to verify accuracy

In [12]:
def add_target_column_to_result_dataframe(result):
    df = result.obs
    y=df['cell_labels']
    encoder = LabelEncoder()
    y_encoded1 = encoder.fit_transform(y)
    df_y = pd.DataFrame(data=y_encoded1, columns=['target'],index=df.index)
    result.obs = pd.concat([df, df_y], axis=1)
    return result

result_target = add_target_column_to_result_dataframe(result)

Turning tensors into numpy arrays to verify accuracy

In [13]:
def create_numpy_array_of_predictions(predictions, prediction_probabilities, data):
    predictions_np = np.zeros((data.shape[0]))
    predictions_probs_np = np.zeros((data.shape[0], number_of_classes))

    y_true = np.zeros((data.shape[0]))
    for i in range(data.shape[0]):
        cluster = data['st_label'].iloc[i]
        predictions_np[i] = predictions[cluster]
        predictions_probs_np[i, :] = prediction_probabilities[cluster, :].numpy()
        y_true[i] = data['target'].iloc[i]

    return predictions_np, predictions_probs_np, y_true

predictions_np, predictions_probs_np, y_true = create_numpy_array_of_predictions(predictions, prediction_probabilities, result_target.obs)

In [14]:
predictions_probs_np.shape

(106445, 14)

Evaluating the model

In [15]:
def evaluate_model(y_predicted, y_predicted_proba, y_true):
    accuracy = accuracy_score(y_true, y_predicted)
    print("Accuracy:", accuracy)

    macro_f1 = f1_score(y_true, y_predicted, average='macro')
    print("Macro F1 score:", macro_f1)

    auc = roc_auc_score(y_true, y_predicted_proba, multi_class='ovr')
    print("AUC score (OvR):", auc)

    average_precision = average_precision_score(y_true, y_predicted_proba)
    print("Average Precision score:", average_precision)
    
    return accuracy, macro_f1, auc, average_precision

In [16]:
accuracy, macro_f1, auc, average_precision = evaluate_model(predictions_np, predictions_probs_np, y_true)

Accuracy: 0.7149983559584762
Macro F1 score: 0.3260337774988855
AUC score (OvR): 0.8419054114072037
Average Precision score: 0.28285268441163797
